<a href="https://colab.research.google.com/github/subikkshas/DA6401/blob/main/DLass1q3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from keras.datasets import fashion_mnist
import numpy as np
from  matplotlib import pyplot as plt
import time
import math
from sklearn.model_selection import train_test_split
from tqdm.auto import tqdm

29515/29515 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
26421880/26421880 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
5148/5148 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
4422102/4422102 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
Train Dataset Shape:  (54000, 28, 28)
Train Target Vector Shape:  (54000,)
Test Dataset Shape: (10000, 28, 28)
Test Target Vector Shape (10000,)
Validation Dataset Shape: (6000, 28, 28)
Validation Target Vector Shape (6000,)


In [5]:

(X, y), (X_test, y_test) = fashion_mnist.load_data()

X = X / 255.0
X_test = X_test / 255.0

X = X.reshape(X.shape[0], -1)
X_test = X_test.reshape(X_test.shape[0], -1)

def one_hot(y, num_classes=10):
    return np.eye(num_classes)[y]

y = one_hot(y)
y_test = one_hot(y_test)

X_train, X_validation, y_train, y_validation = train_test_split(X, y, test_size=0.2, random_state=42)

print("X_train Shape:", X_train.shape)
print("X_validation Shape:", X_validation.shape)
print("X_test Shape:", X_test.shape)
print("Sample X_train Data:", X_train[:5])
print("Sample y_train Data:", y_train[:5])

X_train Shape: (48000, 784)
X_validation Shape: (12000, 784)
X_test Shape: (10000, 784)
Sample X_train Data: [[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
Sample y_train Data: [[0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]]


In [6]:
def relu(x):
    return np.maximum(0, x)

def relu_derivative(x):
    return (x > 0).astype(float)

def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def sigmoid_derivative(x):
    return sigmoid(x) * (1 - sigmoid(x))

def tanh(x):
    return np.tanh(x)

def tanh_derivative(x):
    return 1 - np.tanh(x) ** 2

def softmax(x):
    exp_x = np.exp(x - np.max(x, axis=1, keepdims=True))
    return exp_x / np.sum(exp_x, axis=1, keepdims=True)

def cross_entropy_loss(y_pred, y_true):
    return -np.mean(np.sum(y_true * np.log(y_pred + 1e-9), axis=1))



In [7]:
class NeuralNetwork:
    def __init__(self, input_size=784, hidden_layers=[128, 64], output_size=10, learning_rate=0.01, optimizer='sgd', activation='relu'):
        self.layers = []
        self.biases = []
        self.learning_rate = learning_rate
        self.optimizer = optimizer
        self.activation = activation
        self.momentum = 0.9
        self.beta1 = 0.9
        self.beta2 = 0.999
        self.epsilon = 1e-8
        self.velocities = []
        self.squared_gradients = []
        self.m_t = []
        self.v_t = []

        layer_sizes = [input_size] + hidden_layers + [output_size]
        for i in range(len(layer_sizes) - 1):
            self.layers.append(np.random.randn(layer_sizes[i], layer_sizes[i+1]) * 0.01)
            self.biases.append(np.zeros((1, layer_sizes[i+1])))
            self.velocities.append(np.zeros_like(self.layers[i]))
            self.squared_gradients.append(np.zeros_like(self.layers[i]))
            self.m_t.append(np.zeros_like(self.layers[i]))
            self.v_t.append(np.zeros_like(self.layers[i]))

    def activate(self, x):
        if self.activation == 'relu':
            return relu(x)
        elif self.activation == 'sigmoid':
            return sigmoid(x)
        elif self.activation == 'tanh':
            return tanh(x)
        return x

    def activation_derivative(self, x):
        if self.activation == 'relu':
            return relu_derivative(x)
        elif self.activation == 'sigmoid':
            return sigmoid_derivative(x)
        elif self.activation == 'tanh':
            return tanh_derivative(x)
        return np.ones_like(x)

    def forward(self, x):
        self.a = [x]
        self.z = []

        for i in range(len(self.layers) - 1):
            z = np.dot(self.a[-1], self.layers[i]) + self.biases[i]
            self.z.append(z)
            self.a.append(self.activate(z))

        final_z = np.dot(self.a[-1], self.layers[-1]) + self.biases[-1]
        self.z.append(final_z)
        self.a.append(softmax(final_z))

        return self.a[-1]

    def update_weights(self, dw, db, i):
        if self.optimizer == 'sgd':
            self.layers[i] -= self.learning_rate * dw
            self.biases[i] -= self.learning_rate * db

    def backward(self, y_true):
        m = y_true.shape[0]
        dz = self.a[-1] - y_true

        for i in reversed(range(len(self.layers))):
            dw = np.dot(self.a[i].T, dz) / m
            db = np.sum(dz, axis=0, keepdims=True) / m

            self.update_weights(dw, db, i)

            if i > 0:
                dz = np.dot(dz, self.layers[i].T) * self.activation_derivative(self.z[i-1])

    def train(self, X_train, y_train, epochs=10, batch_size=32):
        for epoch in range(epochs):
            for i in range(0, len(X_train), batch_size):
                x_batch = X_train[i:i+batch_size]
                y_batch = y_train[i:i+batch_size]

                y_pred = self.forward(x_batch)
                self.backward(y_batch)

            y_pred_train = self.forward(X_train)
            loss = cross_entropy_loss(y_pred_train, y_train)
            accuracy = np.mean(np.argmax(y_pred_train, axis=1) == np.argmax(y_train, axis=1))

            print(f"Epoch {epoch+1}: Loss = {loss:.4f}, Accuracy = {accuracy:.4f}")

nn = NeuralNetwork(hidden_layers=[128, 64], learning_rate=0.01, optimizer='adam', activation='relu')
nn.train(X_train, y_train, epochs=10, batch_size=32)

Epoch 1: Loss = 2.3026, Accuracy = 0.1082
Epoch 2: Loss = 2.3026, Accuracy = 0.1082
Epoch 3: Loss = 2.3026, Accuracy = 0.1082
Epoch 4: Loss = 2.3026, Accuracy = 0.1082
Epoch 5: Loss = 2.3026, Accuracy = 0.1082
Epoch 6: Loss = 2.3026, Accuracy = 0.1082
Epoch 7: Loss = 2.3026, Accuracy = 0.1082
Epoch 8: Loss = 2.3026, Accuracy = 0.1082
Epoch 9: Loss = 2.3026, Accuracy = 0.1082
Epoch 10: Loss = 2.3026, Accuracy = 0.1082
